In [16]:
import sqlite3
import pandas as pd

In [17]:
con = sqlite3.connect(':memory:')

In [19]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [20]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [5]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [77]:
pd.read_sql(
    '''
    SELECT CustomerName,Country,Address
    FROM Customers
    where 1=1
        and (country in ('Germany','France')
            or city in ('Madrid'))
    order by 3,2,1
    ''',
    con,
)

,CustomerName,Country,Address
0,La maison d'Asie,France,1 rue Alsace-Lorraine
1,Bon app',France,"12, rue des Bouchers"
2,Folies gourmandes,France,"184, chaussée de Tournai"
3,Victuailles en stock,France,"2, rue du Commerce"
4,Blondel père et fils,France,"24, place Kléber"
5,Spécialités du monde,France,"25, rue Lauriston"
6,Paris spécialités,France,"265, boulevard Charonne"
7,France restauration,France,"54, rue Royale"
8,Vins et alcools Chevalier,France,59 rue de l'Abbaye
9,La corne d'abondance,France,"67, avenue de l'Europe"


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [78]:
pd.read_sql(
    '''
    SELECT Country,count(customername) as cntCustomer
    FROM Customers
    group by 1
    order by 2 desc
    limit 3
    ''',
    con,
)

,Country,cntCustomer
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [7]:
pd.read_sql(
    '''
    select ShipperName,OrderDate from (
        SELECT *,
            row_number() over (order by date(OrderDate)) as rn
        FROM Shippers sh
        join Orders o on o.ShipperID=sh.ShipperID
    ) a
    where rn=10
    
    ''',
    con,
)

,ShipperName,OrderDate
0,Speedy Express,07.08.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [12]:
# проверка
pd.read_sql(
    '''
    SELECT o.OrderID,
        sum(p.Price * od.Quantity) as Cost
    FROM Orders o
    join OrderDetails od on o.OrderID=od.OrderID
    join Products p on p.ProductID=od.ProductID
    group by 1
    order by 2 desc
    limit 1
    ''',
    con,
)

,OrderID,Cost
0,10372,15353.6


In [24]:
pd.read_sql(
    '''
    select OrderID,ProductName,Price from (
    select OrderID,ProductName,Price,
        rank() over(order by cost desc) as rnk
    from (
            SELECT o.OrderID,ProductName,Price,Quantity,
                sum(p.Price * od.Quantity)over (partition by o.OrderID ) as Cost
            FROM Orders o
            join OrderDetails od on o.OrderID=od.OrderID
            join Products p on p.ProductID=od.ProductID
        ) a
        ) a
        where rnk=1
    ''',
    con,
)

,OrderID,ProductName,Price
0,10372,Sir Rodney's Marmalade,81.0
1,10372,Côte de Blaye,263.5
2,10372,Camembert Pierrot,34.0
3,10372,Mozzarella di Giovanni,34.8


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [29]:
pd.read_sql(
    '''
    select OrderID,ProductName,Quantity from (
    select OrderID,ProductName,Quantity,
        rank() over(order by cnt_product desc) as rnk
    from (
            SELECT o.OrderID,ProductName,Price,Quantity,
                sum(od.Quantity)over (partition by p.ProductID ) as cnt_product
            FROM Orders o
            join OrderDetails od on o.OrderID=od.OrderID
            join Products p on p.ProductID=od.ProductID
        ) a
        ) a
        where rnk=1
        order by 1

    ''',
    con,
)

,OrderID,ProductName,Quantity
0,10253,Gorgonzola Telino,20
1,10272,Gorgonzola Telino,40
2,10273,Gorgonzola Telino,15
3,10325,Gorgonzola Telino,4
4,10335,Gorgonzola Telino,25
5,10342,Gorgonzola Telino,56
6,10356,Gorgonzola Telino,30
7,10359,Gorgonzola Telino,70
8,10363,Gorgonzola Telino,20
9,10374,Gorgonzola Telino,30


In [31]:
# проверка
pd.read_sql(
    '''
            SELECT ProductName,sum (Quantity)
            from OrderDetails od 
            join Products p on p.ProductID=od.ProductID
            group by 1
        order by 2 desc
        limit 1
    ''',
    con,
)

,ProductName,sum (Quantity)
0,Gorgonzola Telino,458


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [45]:
#удалить
pd.read_sql(
    '''
    select SupplierID,SupplierName,Country,ContactName,Phone,
        rank() over (order by cnt_orders desc) as rnk
        from (
    SELECT distinct s.SupplierID,s,SupplierName,s.Country,s.ContactName,s.Phone,
        count(o.OrderID) over (partition by s.SupplierID) as cnt_orders
    FROM Suppliers s
    join Products p on s.SupplierID=p.SupplierID
    join OrderDetails od on p.ProductID=od.ProductID
    join Orders o on o.OrderID=od.OrderID
    ) a
    order by 5
    limit 5

    ''',
    con,
)

,SupplierID,Country,ContactName,Phone,rnk
0,12,Germany,Martin Bein,(069) 992755,1
1,7,Australia,Ian Devling,(03) 444-2343,2
2,8,UK,Peter Wilson,(161) 555-4448,3
3,15,Norway,Beate Vileid,(0)2-953010,4
4,14,Italy,Elio Rossi,(0544) 60323,5


In [51]:
pd.read_sql(
    '''
    select SupplierName,Country,ContactName,Phone from (
        SELECT s.SupplierName,s.Country,s.ContactName,s.Phone,
            count(o.OrderID) as cnt_orders
        FROM Suppliers s
        join Products p on s.SupplierID=p.SupplierID
        join OrderDetails od on p.ProductID=od.ProductID
        join Orders o on o.OrderID=od.OrderID
        group by 1,2,3,4
        order by 5 desc
        limit 5
    )a
    ''',
    con,
)

,SupplierName,Country,ContactName,Phone
0,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
1,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
2,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
3,Norske Meierier,Norway,Beate Vileid,(0)2-953010
4,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [57]:
pd.read_sql(
    '''
    SELECT Country,CategoryName,
        sum(od.Quantity*Price) as cost
    FROM Categories c
    join Products p on c.CategoryID=p.CategoryID
    join OrderDetails od on p.ProductID=od.ProductID
    join Orders o on od.OrderID=o.OrderID
    join Customers cm on cm.CustomerID=o.CustomerID
    where 1=1
        and country = 'Brazil'
    group by 1,2
    order by 3 desc
    limit 1
    ''',
    con,
)

,Country,CategoryName,cost
0,Brazil,Beverages,13690.0


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [68]:
pd.read_sql(
    '''
    select country,max(cost)-min(cost) as diff_cost from (
    SELECT o.OrderID,country,
        sum(p.Price * od.Quantity) as Cost
    FROM Orders o
    join OrderDetails od on o.OrderID=od.OrderID
    join Products p on p.ProductID=od.ProductID
    join Customers cm on cm.CustomerID=o.CustomerID
    where country='USA'
    group by 1,2
    ) a
    group by 1
    ''',
    con,
)

,country,diff_cost
0,USA,7638.45


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [58]:
pd.read_sql(
    '''
    select Order_cnt, FirstName|| ' ' || LastName from (
    SELECT distinct LastName,FirstName,BirthDate ,
           dense_rank() over(order by date(substr(BirthDate,7,4)|| '-'||substr(BirthDate,4,2)|| '-'||substr(BirthDate,1,2)) desc) as rnk
           ,count(o.orderID) over (partition by e.EmployeeID) as order_cnt
    FROM Employees e
    join Orders o on e.EmployeeID=o.EmployeeID
   ) a
   where rnk<=3
    ''',
    con,
)

,order_cnt,FirstName|| ' ' || LastName
0,6,Anne Dodsworth
1,29,Nancy Davolio
2,31,Janet Leverling


10. Сколько банок крабового мяса всего было заказано.

In [76]:
pd.read_sql(
    '''
    SELECT ProductName,sum(substr(unit,1,instr(unit,' ')-1)*Quantity) as cnt_crab 
    FROM products p
    join OrderDetails od on p.ProductID=od.ProductID
    where ProductName like '%Crab%Meat%'
    group by 1
    ''',
    con,
)

,ProductName,cnt_crab
0,Boston Crab Meat,6144
